## DarkSky API pull for fire post clustered data
### Main columns needed are latitude, longitude, datetime

In [47]:
import pandas as pd
import datetime
import numpy as np

import config

from darksky import forecast
from datetime import datetime as dt
from datetime import timedelta
from datetime import date
from sqlalchemy import create_engine

In [48]:
# create sqlalchemy engine
engine = create_engine("mysql+mysqlconnector://{user}:{password}@{host}/{dbname}"
                       .format(user=config.db_user,
                               password=config.db_pass,
                               dbname=config.db_name,
                               host=config.db_host))

In [49]:
RAPIDAPI_KEY  = config.darksky_api1
RAPIDAPI_KEY2  = config.darksky_api2

In [50]:
def weather_lookup(df, key=RAPIDAPI_KEY, days_before=0, days_after=0):
    data = []
    for index, row in df.iterrows():
        ts = row['datetime'].isoformat() 
        lat = row['latitude']
        lon = row['longitude']
        weather = forecast(key, lat, lon, time=ts)
        w_dict = weather['currently']
        w_dict['timestamp'] = ts
        w_dict['latitude'] = lat
        w_dict['longitude'] = lon
        data.append(w_dict)
    return data


In [51]:
filename = "fire_lat_lon_data/Emissions2010_DBScan_Clusters.csv"

In [52]:
firedata = pd.read_csv(filename)

In [53]:
firedata['datetime'] = (pd.to_datetime(firedata.year, format='%Y') + 
                       firedata['doy'].apply(np.ceil).apply(
                           lambda x: pd.Timedelta(x, unit='D')) + timedelta(hours=12))

In [54]:
firedata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8237 entries, 0 to 8236
Data columns (total 24 columns):
Unnamed: 0             8237 non-null int64
id                     8237 non-null float64
year                   8237 non-null float64
doy                    8237 non-null float64
longitude              8237 non-null float64
latitude               8237 non-null float64
grid10k                8237 non-null float64
covertype              8237 non-null float64
fuelcode               8237 non-null float64
area_burned            8237 non-null float64
prefire_fuel           8237 non-null float64
consumed_fuel          8237 non-null float64
ECO2                   8237 non-null float64
ECO                    8237 non-null float64
ECH4                   8237 non-null float64
EPM2.5                 8237 non-null float64
cwd_frac               8237 non-null float64
duff_frac              8237 non-null float64
fuel_moisture_class    8237 non-null float64
burn_source            8237 non-null flo

In [55]:
firedata1000 = firedata[:]

In [10]:
firedata2000 = firedata[1000:2000]

In [15]:
firedata3000 = firedata[2000:]

In [56]:
weather_df1.head()

,apparentTemperature,cloudCover,dewPoint,humidity,icon,latitude,longitude,precipAccumulation,precipIntensity,precipProbability,...,pressure,summary,temperature,time,timestamp,uvIndex,visibility,windBearing,windGust,windSpeed
0,80.35,0.0,52.66,0.38,clear-day,41.5250,-121.1583,NaN,0.0007,0.06,...,1013.1,Clear,80.35,1249239600,2009-08-02T12:00:00,9.0,10.000,202.0,8.60,1.99
1,65.15,NaN,43.74,0.46,NaN,42.5251,-115.5344,NaN,0.0000,0.00,...,NaN,NaN,65.15,1250272800,2009-08-14T12:00:00,NaN,NaN,320.0,28.07,18.68
2,79.10,0.0,32.34,0.18,clear-day,41.5870,-120.9057,NaN,0.0000,0.00,...,1014.9,Clear,79.10,1249412400,2009-08-04T12:00:00,9.0,9.932,201.0,13.58,8.73
3,53.52,NaN,31.46,0.43,rain,42.0714,-118.3545,NaN,0.0053,0.52,...,NaN,Possible Drizzle,53.52,1249585200,2009-08-06T12:00:00,NaN,NaN,143.0,5.75,1.92
4,70.43,NaN,48.22,0.45,NaN,43.6058,-106.7566,NaN,0.0000,0.00,...,NaN,NaN,70.43,1246989600,2009-07-07T12:00:00,NaN,NaN,114.0,14.93,6.67


In [57]:
weather_df1 = pd.DataFrame(weather_lookup(firedata1000,RAPIDAPI_KEY))

In [ ]:
#weather_df2 = pd.DataFrame(weather_lookup(firedata4000,RAPIDAPI_KEY2))

In [58]:
db_columns = ['apparentTemperature', 'cloudCover', 'dewPoint',
       'humidity', 'icon', 'latitude', 'longitude', 'precipAccumulation',
       'precipIntensity', 'precipProbability', 'precipType', 'pressure',
       'summary', 'temperature', 'time', 'timestamp', 'uvIndex', 'visibility',
       'windBearing', 'windGust', 'windSpeed']

In [59]:
weather_df1 = weather_df1[db_columns]

In [63]:
#weather_df2 = weather_df2[db_columns]

In [60]:
weather_df1.to_sql('weather_loading', con = engine, if_exists = 'append', chunksize = 100000)

In [ ]:
#weather_df2.to_sql('weather_loading', con = engine, if_exists = 'append', chunksize = 100000)

In [61]:
query = """SELECT * FROM weather_loading;
"""

In [62]:
weather_all_sql = pd.read_sql(query, engine)

In [63]:
#weather_all_sql = weather_all_sql.reset_index()[db_columns]

In [64]:
#weather_all_sql.to_sql('weather_loading', con = engine, if_exists = 'replace', chunksize = 100000)

In [65]:
weather_all_sql.to_csv('weather_40000.csv')

In [66]:
weather_all_sql.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39050 entries, 0 to 39049
Data columns (total 22 columns):
index                  39050 non-null int64
apparentTemperature    38511 non-null float64
cloudCover             30509 non-null float64
dewPoint               38277 non-null float64
humidity               38268 non-null float64
icon                   29962 non-null object
latitude               39050 non-null float64
longitude              39050 non-null float64
precipAccumulation     181 non-null float64
precipIntensity        36927 non-null float64
precipProbability      36927 non-null float64
precipType             4138 non-null object
pressure               23500 non-null float64
summary                30214 non-null object
temperature            38511 non-null float64
time                   39050 non-null int64
timestamp              39050 non-null object
uvIndex                30509 non-null float64
visibility             31139 non-null float64
windBearing            37961